In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
import hashlib

In [17]:
load_dotenv()
data_path = os.getenv("DATA_8")

df = pd.read_csv(data_path)

In [18]:
num_df = df.select_dtypes(include=['number'])
negative_counts = (num_df < 0).sum()

neg = negative_counts[negative_counts > 0]

neg.to_csv(r"C:\Users\user\rde-data\negative_columns_full8.csv",
           header=['negative_count'],
           encoding='utf-8-sig')


In [16]:
def detect_outliers_iqr(df):
    outlier_summary = {}

    for col in df.select_dtypes(include=[np.number]).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR

        outliers = ((df[col] < lower) | (df[col] > upper)).sum()
        outlier_summary[col] = outliers

    outlier_df = pd.DataFrame.from_dict(outlier_summary, orient='index', columns=['outlier_count'])
    return outlier_df.sort_values('outlier_count', ascending=False)

# 실행
outlier_result = detect_outliers_iqr(df)
outlier_result.head(20)   # 이상치 많은 상위 20개 컬럼 출력

,outlier_count
현금흐름,18457
계속사업이익,18150
OCF/매출액비용,18099
영업활동현금흐름,18087
재무활동현금흐름,17657
투자활동현금흐름,16136
차입금/EBITDA,13668
법인세,13608
EBIT,13469
당기순이익,12653
